In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-09 19:11:14--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.45MB/s    in 0.2s    

2021-08-09 19:11:15 (5.45 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Luggage_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9CO86UUJCAW5|          3|            0|          0|   N|                Y|
|R3PR8X6QGVJ8B1|          5|            0|          0|   N|                Y|
|R39BO2819ABUPF|          4|            0|          0|   N|                Y|
|R3ADL7V6EGGEEP|          4|            0|          0|   N|                Y|
|R1OXYPBPLVRMI5|          5|            0|          0|   N|                Y|
|R1WYM8Z5ATQ98O|          3|            0|          0|   N|                Y|
|R3LCIANTN1H9EC|          4|            1|          1|   N|                Y|
|R3U2M23N1P0KQ6|          5|            0|          0|   N|                Y|
|R29MB6N7HB6NZI|          1|            2|          2|   N|                Y|
| RGEQ6DGRG7DQG|          5|            0|          0|   N|     

In [6]:
vine_df_filtered = vine_df.filter("total_votes>=20")
vine_df_filtered.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
| RYIU6GIB4TYT2|          5|           30|         30|   N|                N|
| RNXGKYVCJDDL7|          5|           28|         28|   N|                Y|
|R1E8V7J0K61754|          5|           18|         20|   N|                Y|
| RK52L3B5733BH|          5|           22|         22|   N|                N|
|R213PW7RSI6Z7N|          5|           48|         50|   N|     

In [7]:
vine_helpful_df = vine_df_filtered.filter(vine_df_filtered["helpful_votes"]/vine_df_filtered["total_votes"]>=0.5)
vine_helpful_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
| RYIU6GIB4TYT2|          5|           30|         30|   N|                N|
| RNXGKYVCJDDL7|          5|           28|         28|   N|                Y|
|R1E8V7J0K61754|          5|           18|         20|   N|                Y|
| RK52L3B5733BH|          5|           22|         22|   N|                N|
|R213PW7RSI6Z7N|          5|           48|         50|   N|     

In [8]:
vine_paid= vine_helpful_df.filter("vine=='Y'")
vine_paid.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RPJ7C4HP3BHXN|          5|           25|         29|   Y|                N|
|R3SB08XK0M7993|          4|           44|         49|   Y|                N|
|R26X1DACMCN3YF|          3|           16|         21|   Y|                N|
|R17HKKDE9ZRVOO|          4|           31|         35|   Y|                N|
|R1R7GNSCMO8U7I|          4|           24|         28|   Y|                N|
|R2T83WIPG3IX5D|          4|           72|         76|   Y|                N|
| RD006IWFCUA5A|          5|          122|        133|   Y|                N|
| REOQCBUVUPXP1|          4|           23|         27|   Y|                N|
|R2336R80ZEOKTX|          5|           63|         63|   Y|                N|
| RC1ZXE5GVL9TA|          4|           20|         23|   Y|     

In [9]:
vine_unpaid= vine_helpful_df.filter("vine=='N'")
vine_unpaid.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
| RYIU6GIB4TYT2|          5|           30|         30|   N|                N|
| RNXGKYVCJDDL7|          5|           28|         28|   N|                Y|
|R1E8V7J0K61754|          5|           18|         20|   N|                Y|
| RK52L3B5733BH|          5|           22|         22|   N|                N|
|R213PW7RSI6Z7N|          5|           48|         50|   N|     

In [10]:
vine_paid.count()

21

In [11]:
vine_paid.filter("star_rating=5").count()

10

In [12]:
vine_unpaid.count()

6690

In [13]:
vine_unpaid.filter("star_rating=5").count()

3448

In [16]:
vine_paid_percent = ((vine_paid.filter("star_rating=5").count()/vine_paid.count())*100)
vine_paid_percent

47.61904761904761

In [17]:
vine_unpaid_percent = ((vine_unpaid.filter("star_rating=5").count()/vine_unpaid.count())*100)
vine_unpaid_percent

51.53961136023917